In [ ]:
#Predicts freezing using leave-one-animal-out linear regression.

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import LeaveOneOut
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import matplotlib.lines as mlines

# -------------------------------
# 1) Load Data
# -------------------------------
features_path = r"path"
labels_path = r"path"

features_df = pd.read_excel(features_path)
labels_df = pd.read_excel(labels_path)

# -------------------------------
# 2) Define Features
# -------------------------------
features_first600 = [
    "Firing rate first 600", "mISI (s) first 600", "maxISI (s) first 600",
    "minISI (s) first 600", "CVISI first 600", "PC1score whole", "PC2score whole"
]

features_last600 = [
    "Firing rate last 600", "mISI (s) last 600", "maxISI (s) last 600",
    "minISI (s) last 600", "CVISI last 600", "PC1score whole", "PC2score whole"
]

# -------------------------------
# 3) Aggregate Neuron Features by Animal
# -------------------------------
agg_features_first600 = features_df.groupby("Unique ID")[features_first600].mean().reset_index()
agg_features_last600 = features_df.groupby("Unique ID")[features_last600].mean().reset_index()

# Merge aggregated features with freezing labels
merged_first600 = pd.merge(agg_features_first600, 
                           labels_df[["UniqueID", "Group", "Percentage Freezing First 600"]],
                           left_on="Unique ID", right_on="UniqueID", how="inner")

merged_last600 = pd.merge(agg_features_last600, 
                          labels_df[["UniqueID", "Group", "Percentage Freezing Last 600"]],
                          left_on="Unique ID", right_on="UniqueID", how="inner")

# Drop NaN values (if any)
merged_first600.dropna(subset=features_first600 + ["Percentage Freezing First 600"], inplace=True)
merged_last600.dropna(subset=features_last600 + ["Percentage Freezing Last 600"], inplace=True)

# -------------------------------
# 4) Define X and y for Each Segment
# -------------------------------
X_first600 = merged_first600[features_first600].values
y_first600 = merged_first600["Percentage Freezing First 600"].values
groups_first600 = merged_first600["Group"].values  # Store group info

X_last600 = merged_last600[features_last600].values
y_last600 = merged_last600["Percentage Freezing Last 600"].values
groups_last600 = merged_last600["Group"].values  # Store group info

# Standardize features
scaler_first600 = StandardScaler().fit(X_first600)
X_first600 = scaler_first600.transform(X_first600)

scaler_last600 = StandardScaler().fit(X_last600)
X_last600 = scaler_last600.transform(X_last600)

# -------------------------------
# 5) Perform LOOCV
# -------------------------------
loo = LeaveOneOut()

# Define colors for groups
group_colors = {0: "black", 1: "#ff69b4", 2: "#008080"}  # Control (black), ELS Veh (pink), ELS ACTH (teal)

def run_loocv_detailed(X, y, groups, segment):
    """Runs LOOCV and generates individual prediction plots without the perfect prediction line."""
    predictions = []
    actuals = []

    # LOOCV Iteration
    for train_index, test_index in loo.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        model = LinearRegression()
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        predictions.append(y_pred[0])
        actuals.append(y_test[0])

    # Compute final LOOCV R² after all folds
    final_r2 = r2_score(actuals, predictions)

    # -------------------------------
    # Individual Prediction Plots
    # -------------------------------
    fig, axes = plt.subplots(nrows=4, ncols=5, figsize=(12, 10))  # 4 rows, 5 columns
    axes = axes.flatten()  # Flatten for iteration

    for fold, (ax, y_test, y_pred, group) in enumerate(zip(axes, actuals, predictions, groups)):
        ax.scatter(y_test, y_pred, c=group_colors[group])
        ax.set_title(f"Fold {fold+1}", fontsize=10)
        ax.set_xlabel("Actual % Freezing", fontsize=8)
        ax.set_ylabel("Predicted % Freezing", fontsize=8)
        ax.tick_params(axis='both', which='major', labelsize=7)  # Reduce tick label size

    # Adjust spacing
    plt.subplots_adjust(hspace=0.5, wspace=0.4)  # Increase spacing between plots
    plt.suptitle(f"LOOCV Individual Predictions - {segment}", fontsize=14)
    plt.show()

    return final_r2, actuals, predictions

# -------------------------------
# 6) Run LOOCV for First and Last 600s
# -------------------------------
r2_loocv_first600, actuals_first600, preds_first600 = run_loocv_detailed(X_first600, y_first600, groups_first600, "First 600s")
r2_loocv_last600, actuals_last600, preds_last600 = run_loocv_detailed(X_last600, y_last600, groups_last600, "Last 600s")

# -------------------------------
# 7) Final Aggregated Plots
# -------------------------------
# Custom legend handles
group0_legend = mlines.Line2D([], [], color='black', marker='o', linestyle='None', markersize=8, label='Group 0')
group1_legend = mlines.Line2D([], [], color='#ff69b4', marker='o', linestyle='None', markersize=8, label='Group 1')
group2_legend = mlines.Line2D([], [], color='#008080', marker='o', linestyle='None', markersize=8, label='Group 2')

# --- First 600s Plot ---
plt.figure(figsize=(6, 5), dpi=600)
for i, group in enumerate(groups_first600):
    plt.scatter(actuals_first600[i], preds_first600[i],
                c=group_colors[group])

plt.xlabel("Actual Freezing % (First Segment)")
plt.ylabel("Predicted Freezing % (First Segment)")
plt.title(f"LOOCV Linear Regression - First 600s (R² = {r2_loocv_first600:.2f})")
plt.legend(handles=[group0_legend, group1_legend, group2_legend], loc='best')
plt.savefig("Linear LOOCV First", dpi=600, bbox_inches='tight')
plt.show()

# --- Last 600s Plot ---
plt.figure(figsize=(6, 5), dpi=600)
for i, group in enumerate(groups_last600):
    plt.scatter(actuals_last600[i], preds_last600[i],
                c=group_colors[group])

plt.xlabel("Actual Freezing % (Last Segment)")
plt.ylabel("Predicted Freezing % (Last Segment)")
plt.title(f"LOOCV Linear Regression - Last 600s (R² = {r2_loocv_last600:.2f})")
plt.legend(handles=[group0_legend, group1_legend, group2_legend], loc='best')
plt.savefig("Linear LOOCV Last", dpi=600, bbox_inches='tight')
plt.show()

# Print final R² results
print("Final LOOCV R² values:")
print(f"First 600s: {r2_loocv_first600:.2f}")
print(f"Last 600s: {r2_loocv_last600:.2f}")


In [ ]:
#Trains one standard linear model (no LOOCV) per segment using all animals and calculates adjusted R².

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import numpy as np

# -------------------------------
# 1) Function to Compute Adjusted R²
# -------------------------------
def adjusted_r2(r2, n, p):
    """Compute adjusted R² given R², number of samples (n), and number of predictors (p)."""
    return 1 - ((1 - r2) * (n - 1) / (n - p - 1))

# -------------------------------
# 2) Standard Linear Regression (No LOOCV)
# -------------------------------
def run_standard_linear_regression(X, y, groups, features, segment):
    """Runs a single linear regression on the full dataset and generates a final prediction plot"""
    
    model = LinearRegression()
    model.fit(X, y)
    y_pred = model.predict(X)

    # Compute R² and Adjusted R²
    r2 = r2_score(y, y_pred)
    adj_r2 = adjusted_r2(r2, X.shape[0], X.shape[1])  # n = samples, p = predictors

    # -------------------------------
    # 3) Plot Predicted vs. Actual Freezing Percentage
    # -------------------------------
    plt.figure(figsize=(6, 5))
    for i, group in enumerate(groups):
        plt.scatter(y[i], y_pred[i], c=group_colors[group], edgecolors='k')

    plt.plot([min(y), max(y)], [min(y), max(y)], 'r--', label='Perfect Prediction')
    plt.xlabel(f"Actual Freezing % ({segment})")
    plt.ylabel(f"Predicted Freezing % ({segment})")
    plt.title(f"Linear Regression - {segment}\n R² = {adj_r2:.2f}")
    plt.legend()
    plt.show()

    return r2, adj_r2

# -------------------------------
# 4) Run Standard Linear Regression for First and Last 600s
# -------------------------------
r2_first600, adj_r2_first600 = run_standard_linear_regression(X_first600, y_first600, groups_first600, features_first600, "First 600s")
r2_last600, adj_r2_last600 = run_standard_linear_regression(X_last600, y_last600, groups_last600, features_last600, "Last 600s")

# Print results
print("Standard Linear Regression Results:")
print(f"First 600s: R² = {r2_first600:.2f}, Adjusted R² = {adj_r2_first600:.2f}")
print(f"Last 600s: R² = {r2_last600:.2f}, Adjusted R² = {adj_r2_last600:.2f}")


In [ ]:
#Performs multiple linear regression using statsmodels, outputs:
#Unstandardized + standardized coefficients
#t-values and p-values
#Adjusted R²
#Export to CSV and publication-ready table

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm

# -------------------------------
# 1) Compute Adjusted R²
# -------------------------------
def adjusted_r2(r2, n, p):
    """Compute adjusted R² given R², number of samples (n), and number of predictors (p)."""
    return 1 - ((1 - r2) * (n - 1) / (n - p - 1))

# -------------------------------
# 2) Multiple Linear Regression with p-values and CSV Output
# -------------------------------
def run_standard_linear_regression(X, y, feature_names, segment, save_path=None):
    """Runs multiple linear regression, computes standardized coefficients, p-values, and saves results to CSV."""
    
    # Standardize X
    scaler_X = StandardScaler()
    X_standardized = scaler_X.fit_transform(X)

    # Add Intercept to X for StatsModels
    X_with_intercept = sm.add_constant(X)

    # Fit Model using StatsModels (for p-values)
    model = sm.OLS(y, X_with_intercept).fit()

    # Extract Values
    unstandardized_coeffs = model.params[1:]  # Coefficients excluding the intercept
    intercept = model.params[0]  # Intercept
    p_values = model.pvalues[1:]  # P-values for coefficients
    std_errors = model.bse[1:]  # Standard errors
    t_values = model.tvalues[1:]  # T-values

    # Compute Standardized Coefficients (Beta)
    y_std = np.std(y, ddof=1)  # Standard deviation of y
    standardized_coeffs = (unstandardized_coeffs * scaler_X.scale_) / y_std

    # Compute R² and Adjusted R²
    r2 = model.rsquared
    adj_r2 = model.rsquared_adj

    # -------------------------------
    # 3) Create Pandas DataFrame for Results
    # -------------------------------
    results_df = pd.DataFrame({
        'Predictor': feature_names,
        'B (Unstandardized)': unstandardized_coeffs,
        'Beta (Standardized)': standardized_coeffs,
        't-value': t_values,
        'p-value': p_values
    })
    
    # Add intercept separately
    intercept_df = pd.DataFrame({'Predictor': ['Intercept (Constant)'], 
                                 'B (Unstandardized)': [intercept], 
                                 'Beta (Standardized)': [None], 
                                 't-value': [None], 
                                 'p-value': [None]})
    
    results_df = pd.concat([results_df, intercept_df], ignore_index=True)

    # -------------------------------
    # 4) Save Results to CSV
    # -------------------------------
    if save_path:
        results_df.to_csv(save_path, index=False)
        print(f"Regression results saved to: {save_path}")

    # -------------------------------
    # 5) Print Table (Copy-Paste Friendly)
    # -------------------------------
    print(f"\n{'-'*90}")
    print(f" Multiple Linear Regression Results - {segment}")
    print(f" Adjusted R²: {adj_r2:.3f}")
    print(f"{'-'*90}")
    print(f"{'Predictor':<25} {'B (Unstandardized)':<20} {'Beta (Standardized)':<20} {'t':<10} {'p-value'}")
    print(f"{'-'*90}")

    for _, row in results_df.iterrows():
        print(f"{row['Predictor']:<25} {row['B (Unstandardized)']:<20.3f} {row['Beta (Standardized)'] if row['Beta (Standardized)'] is not None else '':<20} {row['t-value'] if row['t-value'] is not None else '':<10} {row['p-value'] if row['p-value'] is not None else ''}")

    print(f"{'-'*90}")

    # -------------------------------
    # 6) Plot Predicted vs. Actual Freezing Percentage
    # -------------------------------
    y_pred = model.predict(X_with_intercept)

    plt.figure(figsize=(6, 5))
    plt.scatter(y, y_pred, color='blue', edgecolors='k', alpha=0.7)

    # Perfect Prediction Line
    plt.plot([min(y), max(y)], [min(y), max(y)], 'r--', label='Perfect Prediction')

    plt.xlabel(f"Actual Freezing % ({segment})")
    plt.ylabel(f"Predicted Freezing % ({segment})")
    plt.title(f"Linear Regression - {segment}\n Adjusted R² = {adj_r2:.3f}")
    plt.legend()
    plt.show()

    return adj_r2, standardized_coeffs, p_values

# -------------------------------
# 7) Run Regression for First and Last 600s and Save Results
# -------------------------------
adj_r2_first600, coeffs_first600, pvals_first600 = run_standard_linear_regression(
    X_first600, y_first600, features_first600, "First 600s", save_path="regression_results_first600NO.csv"
)

adj_r2_last600, coeffs_last600, pvals_last600 = run_standard_linear_regression(
    X_last600, y_last600, features_last600, "Last 600s", save_path="regression_results_last600.csv"
)

# Print final results
print("\nFinal Adjusted R² values:")
print(f"First 600s: {adj_r2_first600:.3f}")
print(f"Last 600s: {adj_r2_last600:.3f}")


In [ ]:
# GEE Regression (First 600s) at the Neuron Level
# Predicts freezing behavior while accounting for within-animal neuron clustering

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.genmod.generalized_estimating_equations import GEE
from statsmodels.genmod.families import Gaussian
from statsmodels.genmod.cov_struct import Exchangeable
from patsy import dmatrices

# ---------------------------------------
# 1️⃣ Load Data
# ---------------------------------------
features_path = r"path_to_features.xlsx"  # Neuron-level features
labels_path = r"path_to_labels.xlsx"      # Animal-level labels (freezing %, group)

features_df = pd.read_excel(features_path)
labels_df = pd.read_excel(labels_path)

# ---------------------------------------
# 2️⃣ Merge Neuron-Level Features with Animal Labels
# ---------------------------------------
# Merge by 'Unique ID' to propagate group and freezing label to each neuron
merged_df = pd.merge(
    features_df,
    labels_df[["UniqueID", "Group", "Percentage Freezing First 600"]],
    left_on="Unique ID",
    right_on="UniqueID",
    how="inner"
)

# Drop rows with missing features or labels
merged_df.dropna(subset=["Percentage Freezing First 600"], inplace=True)

# ---------------------------------------
# 3️⃣ Define Features and Formula
# ---------------------------------------
features_first600 = [
    "Firing rate first 600", "mISI (s) first 600", "maxISI (s) first 600",
    "minISI (s) first 600", "CVISI first 600", "PC1score whole", "PC2score whole"
]

# Patsy-safe formula
features_first600_q = [f"Q('{col}')" for col in features_first600]
formula = "Q('Percentage Freezing First 600') ~ " + " + ".join(features_first600_q)

# ---------------------------------------
# 4️⃣ Fit GEE Model
# ---------------------------------------
try:
    model = GEE.from_formula(
        formula=formula,
        groups=merged_df["Unique ID"],  # Grouping by animal
        data=merged_df,
        family=Gaussian(),
        cov_struct=Exchangeable()
    )

    result = model.fit()

    print("GEE Results for First 600s (Neuron-Level):")
    print(result.summary())

    # Store predicted values
    merged_df["Predicted Freezing First 600"] = result.predict()

except Exception as e:
    print(f"Error fitting GEE for First 600s: {e}")

# ---------------------------------------
# 5️⃣ Plot Actual vs. Predicted
# ---------------------------------------
if "Predicted Freezing First 600" in merged_df.columns:
    plt.figure(figsize=(6, 5))
    sns.scatterplot(
        x=merged_df["Percentage Freezing First 600"],
        y=merged_df["Predicted Freezing First 600"],
        hue=merged_df["Group"],
        palette="Set1",
        edgecolor="black"
    )
    plt.plot(
        [merged_df["Percentage Freezing First 600"].min(), merged_df["Percentage Freezing First 600"].max()],
        [merged_df["Percentage Freezing First 600"].min(), merged_df["Percentage Freezing First 600"].max()],
        'r--', label="Perfect Prediction"
    )
    plt.xlabel("Actual Freezing % (First 600s)")
    plt.ylabel("Predicted Freezing % (First 600s)")
    plt.title("GEE Regression (Neuron-Level) - First 600s")
    plt.legend()
    plt.tight_layout()
    plt.savefig("plots/gee_pred_vs_actual_first600.png", dpi=300)
    plt.show()


In [ ]:
# GEE Regression (Last 600s) at the Neuron Level
# Predicts freezing behavior while accounting for within-animal neuron clustering

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.genmod.generalized_estimating_equations import GEE
from statsmodels.genmod.families import Gaussian
from statsmodels.genmod.cov_struct import Exchangeable
from patsy import dmatrices

# ---------------------------------------
# 1️⃣ Load Data
# ---------------------------------------
features_path = r"path_to_features.xlsx"
labels_path = r"path_to_labels.xlsx"

features_df = pd.read_excel(features_path)
labels_df = pd.read_excel(labels_path)

# ---------------------------------------
# 2️⃣ Merge Neuron-Level Features with Animal Labels
# ---------------------------------------
merged_df = pd.merge(
    features_df,
    labels_df[["UniqueID", "Group", "Percentage Freezing Last 600"]],
    left_on="Unique ID",
    right_on="UniqueID",
    how="inner"
)

merged_df.dropna(subset=["Percentage Freezing Last 600"], inplace=True)

# ---------------------------------------
# 3️⃣ Define Features and Formula
# ---------------------------------------
features_last600 = [
    "Firing rate last 600", "mISI (s) last 600", "maxISI (s) last 600",
    "minISI (s) last 600", "CVISI last 600", "PC1score whole", "PC2score whole"
]

features_last600_q = [f"Q('{col}')" for col in features_last600]
formula = "Q('Percentage Freezing Last 600') ~ " + " + ".join(features_last600_q)

# ---------------------------------------
# 4️⃣ Fit GEE Model
# ---------------------------------------
try:
    model = GEE.from_formula(
        formula=formula,
        groups=merged_df["Unique ID"],
        data=merged_df,
        family=Gaussian(),
        cov_struct=Exchangeable()
    )

    result = model.fit()

    print("GEE Results for Last 600s (Neuron-Level):")
    print(result.summary())

    merged_df["Predicted Freezing Last 600"] = result.predict()

except Exception as e:
    print(f"Error fitting GEE for Last 600s: {e}")

# ---------------------------------------
# 5️⃣ Plot Actual vs. Predicted
# ---------------------------------------
if "Predicted Freezing Last 600" in merged_df.columns:
    plt.figure(figsize=(6, 5))
    sns.scatterplot(
        x=merged_df["Percentage Freezing Last 600"],
        y=merged_df["Predicted Freezing Last 600"],
        hue=merged_df["Group"],
        palette="Set1",
        edgecolor="black"
    )
    plt.plot(
        [merged_df["Percentage Freezing Last 600"].min(), merged_df["Percentage Freezing Last 600"].max()],
        [merged_df["Percentage Freezing Last 600"].min(), merged_df["Percentage Freezing Last 600"].max()],
        'r--', label="Perfect Prediction"
    )
    plt.xlabel("Actual Freezing % (Last 600s)")
    plt.ylabel("Predicted Freezing % (Last 600s)")
    plt.title("GEE Regression (Neuron-Level) - Last 600s")
    plt.legend()
    plt.tight_layout()
    plt.savefig("plots/gee_pred_vs_actual_last600.png", dpi=300)
    plt.show()
